<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/ClusterAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##*Clustering by fast search and find of denstiy of peaks*
####Alex Rodriguez and Alessandro Laoi, Science (June 2014) p1492

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default= 'plotly_dark'

In [ ]:
def knee(x,y):
    data = np.c_[x,y]
    line = data[-1]-data[0]
    line_norm = line/np.sqrt(np.sum(line**2))
    norm_data = data - data[0]
    perp_vector = norm_data - line_norm[None,:]*np.dot(norm_data,line_norm)[:,None]
    dist = np.sqrt(np.sum(perp_vector**2,axis=1))
    return np.argmax(dist)

In [47]:
np.random.seed(1234)
c = np.array([[-3,-3, -3], [3,3,3], [-3,3,1], [3,-3,2]])
n = np.random.randint(15,20,c.size)
p=[]
for i in range(c.shape[0]):
    p.append(np.random.normal(loc=c[i,:],scale=1,size=(n[i],3)))
p=np.concatenate(p)

In [48]:

rnd_idx=np.random.permutation(np.arange(p.shape[0]))
p=p[rnd_idx]
fig=make_subplots()
fig.add_scatter3d(x=p[:,0],y=p[:,1],z=p[:,2],text=np.arange(x.size),mode='markers+text',marker_color='rgba(0,0,0,0)', marker_line_color='rgba(100,100,100,100)', marker_line_width=1,
                  textposition='middle center')
fig.update_layout(width=800,height=600)

In [49]:
d = (np.sum(((p[None,:]-p[:,None])**2),axis=-1))**0.5
dc = np.percentile(d,5/x.size*100)
rho = np.sum(np.exp(-(d/dc)),axis=1)
rhoij = rho[None,:]>rho[:,None]
delta=np.min(np.logical_not(rhoij)*np.max(d) + d,axis=1)
nearest_higher_density_idx=np.argmin(np.logical_not(rhoij)*np.max(d) + d,axis=1)
rho_max = np.argmax(rho) 
delta[rho_max]=np.max(d[rho_max,:])
delta_rho = delta*rho
print(np.mean(rho),dc)

5.863141656905826 1.406139213145943


In [35]:
delta_idx = np.argsort(delta_rho)[-1::-1]
n_clusters = knee(np.arange(delta.size), delta_rho[delta_idx])
fig3=make_subplots()
fig3.add_scatter(x=np.arange(delta_idx.size),y=delta_rho[delta_idx],mode='markers')
fig3.update_layout(width=600,height=400)

In [36]:
cluster = np.full_like(x,-1,dtype=np.int)
cluster[delta_idx[:n_clusters]]=np.arange(n_clusters)
assigned_idx=np.where(cluster>-1)[0]
n_assigned = assigned_idx.size
non_cluster_center_idx = np.setdiff1d(np.arange(x.size),delta_idx[:n_clusters])

In [38]:
while assigned_idx.size>0:
    data_idx, cluster_idx = np.nonzero(nearest_higher_density_idx[non_cluster_center_idx,None]==assigned_idx[None,:])
    cluster[non_cluster_center_idx[data_idx]]=cluster[assigned_idx[cluster_idx]]
    assigned_idx=non_cluster_center_idx[data_idx]



In [51]:
colors=np.array(['red','blue','green','white','yellow'])
fig2=make_subplots()
fig2.add_scatter3d(x=p[:,0],y=p[:,1],z=p[:,2],text=np.arange(x.size),mode='text', textfont_color=colors[cluster+1], textposition='middle center')
fig2.add_scatter3d(x=p[delta_idx,0],y=p[delta_idx[:n_clusters],1],z=p[delta_idx[:n_clusters],2],mode='markers',marker_size=10,
                   marker_color='rgba(0,0,0,0)',marker_line_color='red',marker_line_width=1, textposition='middle center')
fig2.update_layout(width=800,height=800, showlegend=False)